In [13]:

from netCDF4 import Dataset, num2date
import pandas as pd
import numpy as np
import xarray as xr
nc_file = 'gennaio.nc'
#ds = xr.open_dataset('a492415c2645b30009ac40c242fa6287.nc', engine='cfgrib')
ds = Dataset(nc_file, mode='r')

In [4]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Try opening the file
ds = Dataset("gennaio")

# List variables
print(ds.variables.keys())

OSError: [Errno -51] NetCDF: Unknown file format: 'gennaio'

In [32]:
import cfgrib

ds = cfgrib.open_dataset("era5land_2022_01.nc")
print(ds)

ValueError: unrecognized engine 'cfgrib' must be one of your download engines: ['scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [23]:
import xarray.backends

print(xarray.backends.list_engines())

{'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [ ]:
#print(ds)
temperature = ds.variables['t2m']
precipitations = ds.variables['tp']*1000
#pressure = ds.
print(temperature)



<class 'netCDF4.Variable'>
float32 t2m(valid_time, latitude, longitude)
    _FillValue: nan
    GRIB_paramId: 167
    GRIB_dataType: fc
    GRIB_numberOfPoints: 1
    GRIB_typeOfLevel: surface
    GRIB_stepUnits: 1
    GRIB_stepType: instant
    GRIB_gridType: regular_ll
    GRIB_uvRelativeToGrid: 0
    GRIB_NV: 0
    GRIB_Nx: 1
    GRIB_Ny: 1
    GRIB_cfName: unknown
    GRIB_cfVarName: t2m
    GRIB_gridDefinitionDescription: Latitude/Longitude Grid
    GRIB_iDirectionIncrementInDegrees: 0.1
    GRIB_iScansNegatively: 0
    GRIB_jDirectionIncrementInDegrees: 0.1
    GRIB_jPointsAreConsecutive: 0
    GRIB_jScansPositively: 0
    GRIB_latitudeOfFirstGridPointInDegrees: 44.34
    GRIB_latitudeOfLastGridPointInDegrees: 44.34
    GRIB_longitudeOfFirstGridPointInDegrees: 12.25
    GRIB_longitudeOfLastGridPointInDegrees: 12.25
    GRIB_missingValue: 3.4028234663852886e+38
    GRIB_name: 2 metre temperature
    GRIB_shortName: 2t
    GRIB_totalNumber: 0
    GRIB_units: K
    long_name: 2 metr

In [12]:
temperature = ds.variables['t2m'][:]       # shape: (time, lat, lon)
time = ds.variables['valid_time']
lat = ds.variables['latitude'][:]
lon = ds.variables['longitude'][:]

# Convert time units to datetime objects
time_units = time.units
time_calendar = time.calendar if hasattr(time, 'calendar') else 'standard'
dates = num2date(time[:], units=time_units, calendar=time_calendar)

# Reshape data: (time, lat, lon) -> flat table
times, lats, lons = np.meshgrid(dates, lat, lon, indexing='ij')
data_flat = {
    'time': times.ravel(),
    'latitude': lats.ravel(),
    'longitude': lons.ravel(),
    't2m': temperature.ravel()
}

# Create DataFrame
df = pd.DataFrame(data_flat)

# Optional: Convert temperature from Kelvin to Celsius
df['t2m'] = df['t2m'] - 273.15

In [5]:
import cdsapi
import os

# Initialize client
client = cdsapi.Client()

# Year you want to download
year = 2022

# Define variables and fixed request parts
dataset = "reanalysis-era5-land"
variables = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "surface_pressure",
    "total_precipitation"
]
times = [f"{hour:02d}:00" for hour in range(24)]  # All 24 hours

# Latitude/Longitude bounds [N, W, S, E] #inserire il valore reale nei primi 2 valori
area = [44.563, 11.844, 44.562, 11.843]

# Create output directory if it doesn't exist
output_dir = f"era5_{year}"
os.makedirs(output_dir, exist_ok=True)

# Loop over each month
for month in range(1, 13):
    month_str = f"{month:02d}"
    
    # Determine days in the month
    if month_str in ['01', '03', '05', '07', '08', '10', '12']:
        days = [f"{day:02d}" for day in range(1, 32)]
    elif month_str in ['04', '06', '09', '11']:
        days = [f"{day:02d}" for day in range(1, 31)]
    else:  # February (leap year check)
        days = [f"{day:02d}" for day in range(1, 30 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 29)]

    # Prepare request
    request = {
        "variable": variables,
        "year": str(year),
        "month": month_str,
        "day": days,
        "time": times,
        "format": "netcdf",
        "area": area
    }

    # Output file name
    filename = os.path.join(output_dir, f"lavezzola_{year}_{month_str}.nc")

    # Download
    print(f"Downloading {filename}...")
    client.retrieve(dataset, request).download(filename)

print("✅ Download completed for all months.")

2025-06-05 14:35:30,363 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-06-05 14:35:30,567 INFO Request ID is 5b6f73de-c2eb-4dff-b651-a997b20d1d52


2025-06-05 14:35:30,911 INFO status has been updated to accepted
2025-06-05 14:47:51,995 INFO status has been updated to running
2025-06-05 14:55:52,769 INFO status has been updated to successful
2025-06-05 14:55:54,463 INFO Request ID is 8d5ffd25-8372-4b7c-8a51-4e89e67d4abd           


2025-06-05 14:55:54,531 INFO status has been updated to accepted
2025-06-05 14:58:46,768 INFO status has been updated to running
2025-06-05 15:08:14,623 INFO status has been updated to successful


2025-06-05 15:08:16,331 INFO Request ID is 101c8e3d-1687-483d-b754-fdcc3adb75ce
2025-06-05 15:08:16,502 INFO status has been updated to accepted
2025-06-05 15:09:06,567 INFO status has been updated to running
2025-06-05 15:18:36,067 INFO status has been updated to successful


2025-06-05 15:18:37,653 INFO Request ID is bf67e3ed-4778-4c04-bec5-8f1b6fa3bf70
2025-06-05 15:18:37,724 INFO status has been updated to accepted
2025-06-05 15:51:04,510 INFO status has been updated to running
2025-06-05 15:57:05,123 INFO status has been updated to successful


...

2025-06-05 18:19:03,607 INFO Request ID is c2d5588c-15e8-4105-8340-326fb664fe4e
2025-06-05 18:19:03,682 INFO status has been updated to accepted
2025-06-05 18:27:23,233 INFO status has been updated to running
2025-06-05 18:35:26,697 INFO status has been updated to successful


2025-06-05 18:35:28,514 INFO Request ID is 1f39e710-42b8-4094-b2d6-9ecfedc0ff7b
2025-06-05 18:35:28,601 INFO status has been updated to accepted
2025-06-05 18:43:48,574 INFO status has been updated to running
2025-06-05 18:53:49,489 INFO status has been updated to successful


2025-06-05 18:53:51,135 INFO Request ID is f7bbe739-d8a4-465d-b0ec-6fbb3136d43b
2025-06-05 18:53:51,209 INFO status has been updated to accepted
2025-06-05 18:54:41,058 INFO status has been updated to running
2025-06-05 18:56:43,152 INFO status has been updated to accepted
2025-06-05 18:58:09,841 INFO status has been updated to running
2025-06-05 19:06:10,651 INFO status has been updated to successful


2025-06-05 19:06:12,668 INFO Request ID is 4d4afec3-8cdf-44ed-8e89-295c8c0b0741
2025-06-05 19:06:12,867 INFO status has been updated to accepted
2025-06-05 19:22:38,134 INFO status has been updated to running
2025-06-05 19:32:39,231 INFO status has been updated to successful
                                                                                          

✅ Download completed for all months.


In [ ]:
import numpy as np

def compute_relative_humidity(temp_c, dewpoint_c):
    """
    Calculate relative humidity (%) from temperature and dew point (in Celsius).
    Uses the Magnus formula.
    """
    a, b = 17.625, 243.04
    rh = 100 * (np.exp((a * dewpoint_c) / (b + dewpoint_c)) /
                np.exp((a * temp_c) / (b + temp_c)))
    return np.clip(rh, 0, 100)  # Ensure RH is within 0-100%

In [ ]:
temp_c = temperature - 273.15
dewpoint_c = dewpoint_k - 273.15
rh = compute_relative_humidity(temp_c, dewpoint_c)